In [1]:
import numpy as np
import math
from scipy.stats import norm

In [2]:
def linear_congruential_generator(N):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = 1

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples

## Test Script:
N = 10
print(linear_congruential_generator(N))

[1.83344819e-05 7.21883555e-01 7.21203668e-01 9.52016746e-01
 7.55345664e-01 2.24823678e-01 9.82692900e-01 5.67537527e-01
 6.55063564e-01 8.17715721e-01]


In [3]:
def inverse_normal_approximation(u):
    """Generates normally distributed realizations from uniform random realizations."""
    
    ## Constants for approximations to inverse normal.
    a0, a1, a2, a3 = 2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637
    b0, b1, b2, b3 = -8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833
    c0, c1, c2, c3 = 0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863
    c4, c5, c6, c7, c8 = 0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364, 0.0000003960315187

    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = -1 * math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4 + r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x

    return x

## Test Script:
u = 0.8
print(inverse_normal_approximation(u))
## Results verified here: https://statisticshelper.com/inverse-normal-distribution-calculator/#answer

0.8416212348979947


In [4]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

## Test Script:
S = 100
K = 100
T = 1
r = 0.03
q = 0.03
sigma = 0.2

print("Call Option Value:", bs_call(S, K, T, r, q, sigma))
print("Put Option Value:", bs_put(S, K, T, r, q, sigma))
print("Call Option Delta:", bs_call_delta(S, K, T, r, q, sigma))
print("Put Option Delta:", bs_put_delta(S, K, T, r, q, sigma))
print("Option Gamma:", bs_gamma(S, K, T, r, q, sigma))
print("Option Vega:", bs_vega(S, K, T, r, q, sigma))
## Verified results are correct here: https://goodcalculators.com/black-scholes-calculator/

Call Option Value: 7.730149359277917
Put Option Value: 7.730149359277917
Call Option Delta: 0.5238735135706436
Put Option Delta: -0.4465720199778645
Option Gamma: 0.019261041336488413
Option Vega: 38.52208267297682


In [5]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * math.exp((r - q - 0.5 * sigma**2) * T + sigma * math.sqrt(T) * zi)

def call_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European call option at time T."""
    return math.exp(-r * T) * max(Si - K, 0)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return math.exp(-r * T) * max(K - Si, 0)

def call_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European call option at time T."""
    indicator = 1 if Si > K else 0
    return indicator * math.exp(-r * T) * Si / S0

def put_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European put option at time T."""
    indicator = -1 if K > Si else 0
    return indicator * math.exp(-r * T) * Si / S0

def call_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European call option at time T."""
    indicator = 1 if Si > K else 0
    return indicator * Si * math.exp(-r * T) * (-sigma * T + math.sqrt(T) * zi)

def put_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European put option at time T."""
    indicator = -1 if K > Si else 0
    return indicator * Si * math.exp(-r * T) * (-sigma * T + math.sqrt(T) * zi)

## Test Script:
S0 = 100
K = 100
T = 1
r = 0.03
q = 0.03
sigma = 0.2
zi = 1

Si = simulated_spot(S0, T, r, q, sigma, zi)
print("Simulated Spot (Si):", Si)
print("Call Option Payoff (Ci):", call_payoff(Si, K, T, r))
print("Put Option Payoff (Pi):", put_payoff(Si, K, T, r))
print("Call Option Delta Estimate (∆i(C)):", call_delta_estimate(Si, S0, K, T, r))
print("Put Option Delta Estimate (∆i(P)):", put_delta_estimate(Si, S0, K, T, r))
print("Call Option Vega Estimate (vegai(C)):", call_vega_estimate(Si, K, T, r, sigma, zi))
print("Put Option Vega Estimate (vegai(P)):", put_vega_estimate(Si, K, T, r, sigma, zi))

Simulated Spot (Si): 119.72173631218101
Call Option Payoff (Ci): 19.138870917977485
Put Option Payoff (Pi): 0.0
Call Option Delta Estimate (∆i(C)): 1.161834242728283
Put Option Delta Estimate (∆i(P)): 0.0
Call Option Vega Estimate (vegai(C)): 92.94673941826265
Put Option Vega Estimate (vegai(P)): 0.0


In [6]:
def HW1_NUM3(S0, K, T, r, q, sigma, z):
    ## Initialize variables.
    N = len(z)
    C_hat = 0
    delta_C_hat = 0
    vega_C_hat = 0
    # P_hat = 0
    # delta_P_hat = 0
    # vega_P_hat = 0
    
    for i in range(len(z)):
        ## Simulate spot price.
        Si = simulated_spot(S0, T, r, q, sigma, z[i])

        ## Simulate call payoff, delta, and vega. Add to running sum.
        C_hat += call_payoff(Si, K, T, r)
        delta_C_hat += call_delta_estimate(Si, S0, K, T, r)
        vega_C_hat += call_vega_estimate(Si, K, T, r, sigma, z[i])

        ## Simulate put payoff, delta, and vega. Add to running sum.
        # P_hat += put_payoff(Si, K, T, r)
        # delta_P_hat += put_delta_estimate(Si, S0, K, T, r)
        # vega_P_hat += put_vega_estimate(Si, K, T, r, sigma, z[i])

    ## Find Monte Carlo estimates.
    C_hat /= N
    delta_C_hat /= N
    vega_C_hat /= N
    # P_hat /= N
    # delta_P_hat /= N
    # vega_P_hat /= N

    ## Output results:
    print("Results for N =", N)
    print("Call Option Value -- Monte Carlo:", C_hat)
    print("Call Option Value -- BS:", bs_call(S0, K, T, r, q, sigma))
    print("sqrt(N)*|C_BS - C_hat|:", math.sqrt(N)*abs(bs_call(S0, K, T, r, q, sigma)-C_hat))
    print("Call Option Delta -- Monte Carlo:", delta_C_hat)
    print("Call Option Delta -- BS:", bs_call_delta(S0, K, T, r, q, sigma))
    print("sqrt(N)*|delta_C_BS - delta_C_hat|:", math.sqrt(N)*abs(bs_call_delta(S0, K, T, r, q, sigma)-delta_C_hat))      
    print("Call Option Vega -- Monte Carlo:", vega_C_hat)
    print("Call Option Vega -- BS:", bs_vega(S0, K, T, r, q, sigma))
    print("sqrt(N)*|vega_C_BS - vega_C_hat|:", math.sqrt(N)*abs(bs_vega(S0, K, T, r, q, sigma)-vega_C_hat))
    # print("Put Option Value:", P_hat)
    # print("Put Option Delta Estimate:", delta_P_hat)
    # print("Put Option Vega Estimate:", vega_P_hat)
    print("\n")

In [7]:
## Table values for #3.

## Given values.
S = 41
K = 42
sigma = 0.25
q = 0.01
r = 0.03
T = 0.75

## Initialize variables. 
N = 10_000

for k in range(10):
    z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
    HW1_NUM3(S0, K, r, q, sigma, T, z)

Results for N = 10000
Call Option Value -- Monte Carlo: 56.89323250951463
Call Option Value -- BS: 57.2654035921242
sqrt(N)*|C_BS - C_hat|: 37.21710826095688
Call Option Delta -- Monte Carlo: 0.988806343993253
Call Option Delta -- BS: 0.9925280548078762
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.3721710814623247
Call Option Vega -- Monte Carlo: -1.0582313788184345
Call Option Vega -- BS: 1.3325099051930655e-09
sqrt(N)*|vega_C_BS - vega_C_hat|: 105.82313801509444


Results for N = 20000
Call Option Value -- Monte Carlo: 56.84534168538109
Call Option Value -- BS: 57.2654035921242
sqrt(N)*|C_BS - C_hat|: 59.40572455524153
Call Option Delta -- Monte Carlo: 0.9883274357519211
Call Option Delta -- BS: 0.9925280548078762
sqrt(N)*|delta_C_BS - delta_C_hat|: 0.5940572439294635
Call Option Vega -- Monte Carlo: -1.1195152162800635
Call Option Vega -- BS: 1.3325099051930655e-09
sqrt(N)*|vega_C_BS - vega_C_hat|: 158.3233604030768


Results for N = 40000
Call Option Value -- Monte Carlo: 56.8063476967322

In [8]:
## Central Finite Difference Formulas
def call_delta_estimate_FDM(Si_plus, Si_minus, ds, K, T, r):
    """Calculate the delta estimate of the European call option at time T using central finite differences."""
    return (call_payoff(Si_plus, K, T, r) - call_payoff(Si_minus, K, T, r))/(2*ds)

def call_gamma_estimate_FDM(Si, Si_plus, Si_minus, ds, K, T, r):
    """Calculate the gamma estimate of the European call option at time T using central finite differences."""
    return (call_payoff(Si_plus, K, T, r) - 2 * call_payoff(Si, K, T, r) + call_payoff(Si_minus, K, T, r))/(ds**2)

In [11]:
def HW1_NUM4(S0, K, T, r, q, sigma, z):
    ## Initialize variables.
    N = len(z)
    delta_C_hat = 0
    gamma_C_hat = 0
    ds = 0.01
    
    for i in range(len(z)):
        ## Simulate spot prices.
        Si = simulated_spot(S0, T, r, q, sigma, z[i])
        Si_plus = simulated_spot(S0 + ds, T, r, q, sigma, z[i])
        Si_minus = simulated_spot(S0 - ds, T, r, q, sigma, z[i])

        ## Simulate call payoff, delta, and vega. Add to running sum.
        delta_C_hat += call_delta_estimate_FDM(Si_plus, Si_minus, ds, K, T, r)
        gamma_C_hat += call_gamma_estimate_FDM(Si, Si_plus, Si_minus, ds, K, T, r)

    ## Find Monte Carlo estimates.
    delta_C_hat /= N
    gamma_C_hat /= N

    ## Output results:
    print("Results for N =", N)
    print("Call Option Delta -- FDM:", delta_C_hat)
    print("Call Option Delta -- BS:", bs_call_delta(S0, K, T, r, q, sigma))
    print("sqrt(N)*|delta_C_BS - delta_C_hat|:", math.sqrt(N)*abs(bs_call_delta(S0, K, T, r, q, sigma)-delta_C_hat))      
    print("Call Option Gamma -- FDM:", gamma_C_hat)
    print("Call Option Gamma -- BS:", bs_gamma(S0, K, T, r, q, sigma))
    print("sqrt(N)*|gamma_C_BS - gamma_C_hat|:", math.sqrt(N)*abs(bs_gamma(S, K, T, r, q, sigma)-gamma_C_hat))
    print("\n")

In [12]:
## Table values for #4.

for k in range(10):
    z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
    HW1_NUM4(S0, K, T, r, q, sigma, z)

Results for N = 10000
Call Option Delta -- FDM: 0.9816567786894038
Call Option Delta -- BS: 0.9925138624199704
sqrt(N)*|delta_C_BS - delta_C_hat|: 1.0857083730566597
Call Option Gamma -- FDM: -6.252776074688882e-13
Call Option Gamma -- BS: 2.88536157849361e-06
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 4.450878768803753


Results for N = 20000
Call Option Delta -- FDM: 0.9808817849693996
Call Option Delta -- BS: 0.9925138624199704
sqrt(N)*|delta_C_BS - delta_C_hat|: 1.6450241689171512
Call Option Gamma -- FDM: -6.714628852932947e-13
Call Option Gamma -- BS: 2.88536157849361e-06
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 6.294493119327263


Results for N = 40000
Call Option Delta -- FDM: 0.9802040362953832
Call Option Delta -- BS: 0.9925138624199704
sqrt(N)*|delta_C_BS - delta_C_hat|: 2.4619652249174395
Call Option Gamma -- FDM: -6.341593916658894e-13
Call Option Gamma -- BS: 2.88536157849361e-06
sqrt(N)*|gamma_C_BS - gamma_C_hat|: 8.901757537609283


Results for N = 80000
Call Option Delta -- FDM: 0